# Clusterização de Notícias

In [14]:
import pandas as pd

Base possui 60000 textos de notícias em português. Os textos foram extraídos da Folha de São Paulo e consiste de um subconjunto da base utilizada no desafio correspondente do Kaggle.

In [15]:
dados_noticias = pd.read_csv("https://www.data2learning.com/repo/dados_news.csv")
dados_noticias.head()

,index,text
0,0,No dia em que deixou a Síria com instruções pa...
1,1,Havia um vazio em Brasília e ele foi ocupado p...
2,2,Espaços vazios de terminais do metrô e de ônib...
3,3,A primeira imagem mostra uma multidão de milha...
4,4,A seletiva francesa para o Mundial de Kazan te...


In [16]:
dados_noticias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   60000 non-null  int64 
 1   text    59999 non-null  object
dtypes: int64(1), object(1)
memory usage: 937.6+ KB


## Pré-processamento

In [17]:
# Imports necessários
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
# import numpy as np

# Criação da lista de textos que serão vetorizados. O método não aceita o dataframe como entrada. 
textos_unicode = dados_noticias['text'].astype('U').values # ok
texts = list(textos_unicode)

# Stopwords em portugues
portuguese = stopwords.words('portuguese')

# Aplicação da Vetorização
tfidf_vectorizer = TfidfVectorizer(max_df=0.8,min_df=0.2, stop_words=portuguese,
                                   use_idf=True, ngram_range=(1,3),
                                   max_features=None)

tfidf_matrix = tfidf_vectorizer.fit_transform(texts)

In [21]:
from sklearn.cluster import KMeans
from sklearn import metrics

k = 15

km = KMeans(n_clusters=k,random_state=42)
km.fit(tfidf_matrix)

score = metrics.silhouette_score(tfidf_matrix, km.labels_, metric='euclidean')

print("Para k=%i o valor Silhoutte foi de %.4f" % (k, score))

Para k=15 o valor Silhoutte foi de 0.0545


In [25]:
print("formato da matriz normal",tfidf_matrix.shape)
print()

array = tfidf_matrix.toarray()

print("tamanho do array", array.size)

formato da matriz normal (60000, 59)

tamanho do array 3540000


Notei que quanto mais aumenta a quantidade de cluster melhora o silhoute, mas como demora muito, escolhi 15 como o numero de clusters, achei razoavel para testar pq os valores foram ficando mais proximos.

Testei várias maneiras para diminuir o tempo do treinamento e da vetorização. Variei parametros como n_gram, max_df e max features. Porém o que mais influenciou no tempo foi max features que tem relação com a  quantidade de colunas, mas trabalhar com 60k linhas dificulta muito para usar outras tecnicas que precisam de um array, portanto só usei o kmeans mesmo pq recebe a matriz inteira sem precisar transformar e não dá problema de estouro de memória.

In [27]:
# Fazendo predict e adicionando-os na tabela

clusters_km = km.fit_predict(tfidf_matrix)

dados_noticias['cluster_km'] = clusters_km
dados_noticias

,index,text,cluster_km
0,0,No dia em que deixou a Síria com instruções pa...,1
1,1,Havia um vazio em Brasília e ele foi ocupado p...,13
2,2,Espaços vazios de terminais do metrô e de ônib...,10
3,3,A primeira imagem mostra uma multidão de milha...,9
4,4,A seletiva francesa para o Mundial de Kazan te...,14
...,...,...,...
59995,59995,O PSOE (Partido Socialista Operário Espanhol) ...,12
59996,59996,"Nesta quarta (25), às 22h, o Palmeiras vai dis...",0
59997,59997,O Palmeiras está em vias de acabar com um jeju...,10
59998,59998,"O especial multimídia ""Boyhood Bolsa Família"",...",10


## Análise dos clusters

In [28]:
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
order_centroids

array([[19, 49,  6, 51,  5, 54, 57,  3, 40, 52, 42, 10, 43, 31, 16, 45,
        25, 39, 36, 11, 41, 44, 20, 32, 12, 38, 14, 56, 37,  2,  9, 33,
         4, 28, 55, 29,  7, 17, 15,  8, 13, 21, 18, 58, 53, 34, 50, 23,
        35,  0, 46, 30, 24, 22, 26, 27, 48,  1, 47],
       [23, 28, 49, 18, 41, 27, 13, 40,  3, 51, 34, 52,  5, 14, 42,  9,
        16, 20, 45, 36,  6,  1, 39,  0, 38, 29, 33, 56, 22, 19, 54, 32,
        48, 17,  4, 35, 50, 12, 43,  8, 30, 55, 26,  7, 15, 46, 21, 47,
        11, 37, 10, 57, 31, 58,  2, 44, 25, 24, 53],
       [45, 28, 18,  1, 13, 52, 24, 34, 41, 27, 51, 11, 49,  3, 26, 54,
        12,  9, 10, 47, 57, 43, 20, 16, 48, 15, 50, 22, 14,  7, 25,  6,
         5, 17, 30, 46,  4, 44,  0, 39, 19, 55,  2, 31, 40, 38, 35, 58,
        33, 23,  8, 42, 32, 56, 37, 36, 29, 21, 53],
       [42, 49, 18, 41, 33, 36, 27, 51, 56,  9, 16,  6,  3, 13, 20, 52,
        38, 34, 29, 23, 21, 22, 28,  5, 40, 14, 54, 35, 39, 30, 32,  0,
        55,  1, 19,  7,  8,  4, 17, 11, 48, 43, 3

In [29]:
# Criação de uma veriável com a lista de palavras que representa as colunas da matriz.
terms = tfidf_vectorizer.get_feature_names()

In [33]:
centroids = []
for i in range(15):
  list_id = order_centroids[i]
  list_terms = [terms[id] for id in list_id[:10]]
  centroids.append({'cluster': i, 'words': ', '.join(list_terms)})

df_centroids = pd.DataFrame(centroids)
df_centroids

,cluster,words
0,0,"diz, segundo, anos, ser, ano, ter, vai, ainda,..."
1,1,"estado, governo, segundo, disse, país, feira, ..."
2,2,"presidente, governo, disse, afirmou, contra, s..."
3,3,"pessoas, segundo, disse, país, menos, onde, fe..."
4,4,"ex, presidente, federal, disse, segundo, afirm..."
5,5,"acordo, feira, segundo, nesta, ainda, disse, a..."
6,6,"ser, pode, sobre, vai, porque, ainda, ter, faz..."
7,7,"paulo, segundo, feira, contra, dia, dois, ser,..."
8,8,"feira, nesta, quarta, contra, após, segunda, s..."
9,9,"disse, afirmou, sobre, ter, segundo, president..."


In [39]:
for n in range(15):
  print("Cluster %i: %s" % (n, list(df_centroids[df_centroids['cluster'] == n]['words'][:5])))
  data_selected = dados_noticias[dados_noticias['cluster_km'] == n]
  for index, row in data_selected.head(10).iterrows():
    print("\tNotícia: %s" % row['text'])
  print()

Cluster 0: ['diz, segundo, anos, ser, ano, ter, vai, ainda, paulo, sobre']
	Notícia: A delação premiada de Sérgio Machado atinge a Camargo Corrêa no momento em que a empreiteira estava prestes a fechar acordo de leniência com o Ministério Público Federal. O prazo para que isso ocorra termina em duas semanas.  MEA CULPA Acionistas e executivos da empreiteira podem aderir à leniência, confessando crimes e irregularidades em troca de imunidade na Justiça. A dúvida é saber se os procuradores aceitariam agora perdoar fatos graves tornados públicos com a delação de Machado.  EM MÃOS Entre outras coisas, Machado diz que recebeu U$ 350 mil (ou o equivalente a R$ 1,5 milhão) em dinheiro vivo de Luiz Nascimento, um dos controladores do grupo, destinados ao PSDB.  A PRÓXIMA Depois da citação a Michel Temer na delação de Sérgio Machado, cresce nos meios jurídicos envolvidos na negociação da delação premiada da Odebrecht a discussão sobre a forma com que o presidente interino aparecerá nas informaç

Apesar das complicações técnicas em relação a quantidade de linhas, a separação até que foi interessante. 

Cluster 0 - notícias relacionadas ao depoimento de alguem entrevistado.

Cluster 1 - notícias relacionadas ao Estado e Gorverno

Cluster 2 - notícias relacionadas ao presidente de algum governo

Cluster 3 - notícias relacionadas a pessoas, ele reconhece bem nomes próprios

Cluster 4 - notícias sobre algum ex presidente de qualquer tipo

Cluster 5 - notícias sobre acordos em sua maioria na semana

Cluster 6 - notícias sobre o que pode ser que aconteça, preditivas de certa forma

Cluster 7 - notícias sobre São Paulo ou algum Paulo

Cluster 8 - notícias sobre algo que aconteceu na semana

Cluster 9 - notícias sobre algo que alguem afirmou ou disse

Cluster 10 - notícias relacionadas aos anos que vem pela frente, pricipalmente 2 a três anos no futuro

Cluster 11 - notícias sobre algo que o Brasil esteja envolvido

Cluster 12 - notícias relacionadas ao governo e ao presidente

Cluster 13 - notícias sobre os órgãos da federação

Cluster 14 - notícias relacionadas ao ano da notícia 